# MultiDoc-QA - Reranking

Author: Monique Monteiro (moniquelouise@gmail.com)

Inspired by https://github.com/neuralmind-ai/visconde/blob/main/iirc_rerank.ipynb

@inproceedings{10.1007/978-3-031-28238-6_44,
author = {Pereira, Jayr and Fidalgo, Robson and Lotufo, Roberto and Nogueira, Rodrigo},
title = {Visconde: Multi-Document QA With&nbsp;GPT-3 And&nbsp;Neural Reranking},
year = {2023},
isbn = {978-3-031-28237-9},
publisher = {Springer-Verlag},
address = {Berlin, Heidelberg},
url = {https://doi.org/10.1007/978-3-031-28238-6_44},
doi = {10.1007/978-3-031-28238-6_44},
booktitle = {Advances in Information Retrieval: 45th European Conference on Information Retrieval, ECIR 2023, Dublin, Ireland, April 2–6, 2023, Proceedings, Part II},
pages = {534–543},
numpages = {10},
location = {Dublin, Ireland}
}

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:

main_dir = "/content/gdrive/MyDrive/Unicamp-aula-11"

## Libraries installation

In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 14.2 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.9 MB/s eta 0:00:00


In [ ]:
! git clone --recursive https://github.com/castorini/pygaggle.git

%cd pygaggle/

Cloning into 'pygaggle'...
remote: Enumerating objects: 1568, done.
remote: Counting objects: 100% (640/640), done.
remote: Compressing objects: 100% (234/234), done.
remote: Total 1568 (delta 531), reused 436 (delta 406), pack-reused 928
Receiving objects: 100% (1568/1568), 514.89 KiB | 1.16 MiB/s, done.
Resolving deltas: 100% (1004/1004), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/pygaggle/tools'...
remote: Enumerating objects: 788, done.        
remote: Counting objects: 100% (545/545), done.        
remote: Compressing objects: 100% (467/467), done.        
remote: Total 788 (delta 101), reused 514 (delta 77), pack-reused 243        
Receiving objects: 100% (788/788), 119.60 MiB | 17.92 MiB/s, done.
Resolving deltas: 100% (185/185), done.
Submodule path 'tools': checked out '808f48711b5e172da6aec8b1855518c8ea65489f'
/content/pygaggle


In [ ]:
! pip install -r requirements.txt 
! pip install -e /content/pygaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 122.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.7/508.7 kB 50.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.4 MB/s eta 0:00:00
     ━━━

## Reranking

In [ ]:

from transformers import AutoTokenizer
from transformers import T5ForConditionalGeneration

import torch

model = T5ForConditionalGeneration.from_pretrained("castorini/monot5-3b-msmarco-10k", torch_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained("t5-3b")

device = torch.device("cuda")
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-3b automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=4096, bias=False)
              (k): Linear(in_features=1024, out_features=4096, bias=False)
              (v): Linear(in_features=1024, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=16384, bias=False)
              (wo): Linear(in_features=16384, out_features=1024, bias=False)
              

In [ ]:

from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5

reranker = MonoT5(model=model)

2023-05-15 23:48:09 [INFO] loader: Loading faiss with AVX2 support.
2023-05-15 23:48:09 [INFO] loader: Successfully loaded faiss with AVX2 support.


In [ ]:
import json

test_set = json.load(open(f"{main_dir}/to_rerank.json",'r'))
     

In [ ]:
from tqdm import tqdm

results = []

for item in tqdm(test_set):
  chosen = []
  for d in item['decomposition']:
    query = Query(d['question'])
    texts = [Text(p['contents'], {'docid': p['id'],"title":p['title'],"text":p['contents']}, 0) for p in d['documents']]
    reranked = reranker.rerank(query, texts)
    reranked.sort(key=lambda x: x.score, reverse=True)
    d['documents'] = [text.metadata for text in reranked]
  results.append(item)

100%|██████████| 50/50 [05:34<00:00,  6.69s/it]


In [ ]:
json.dump(results, open(f"{main_dir}/iirc_reranked.json",'w'))